In [ ]:
%matplotlib inline

import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.image as mpimg
from matplotlib import rcParams
from matplotlib.colors import LinearSegmentedColormap as lsc
import matplotlib
import seaborn as sns


print("matplotlib version: ", matplotlib.__version__)
if not matplotlib.__version__ == "3.1.3":
    print("Use matplotlib 3.1.3 for this script!")
    print("install dependencies with conda:\nconda env create -f htqpcr_env.yml")

print("seaborn version:", sns.__version__)
if not sns.__version__ == "0.10.1":
    print("Use seaborn 0.10.1 for this script!")
    print("install dependencies with conda:\nconda env create -f htqpcr_env.yml")

rcParams.update({'figure.autolayout': True})

abbrev_dict = {
    "Lactobacillus": "L.",
    "Levilactobacillus": "L.",
    "Lacticaseibacillus": "L.",
    "Loigolactobacillus": "L.",
    "Latilactobacillus": "L.",
    "Limosilactobacillus": "L.",
    "Lentilactobacillus": "L.",
    "Lactiplantibacillus": "L.",
    "Leuconostoc": "Ln.",
    "Streptococcus": "S.",
    "Pediococcus": "Pd.",
    "Propionibacterium": "Pr.",
    "Acidipropionibacterium": "A.",
    "Clostridium": "C.",
    "Lactococcus": "Lc.",
    "Enterococcus": "E."
}

def get_assay_dict():
    assay_dict = {}
    assay_species = "labelfiles/assay_species.csv"
    with open(assay_species) as f:
        reader = csv.reader(f)
        for row in reader:
            p = row[0]
            s = abbrev_dict[row[1].split(" ")[0]] + " " + " ".join(row[1].split(" ")[1::])
            assay_dict.update({p: s})
    return assay_dict

if not os.path.isdir("Figures"):
    os.mkdir("Figures")


In [ ]:
# BioMark rawdata

# Annotate BioMark rawdata
# Create labels
df = pd.read_csv("labelfiles/samples_and_assay_names_validation.csv")
samplenames = list(df["Sample_Name"].dropna())
assaynames = list(df["Assay_Name"].dropna())

# add better labels y-axis
assay_dict = get_assay_dict()
ylabel = assaynames 
newylabels = []
replicates = 3
for i, tik in enumerate(ylabel):
    if i % replicates == 0: 
        label = tik.split(" ")[0]
        label = assay_dict[label]
        newylabels.append(label)

xrange = np.arange(5, 1191, 7)
x_ticks = np.arange(9, 1191, 14)
yticks = np.arange(9, 505, 21)

posdict = {}
l1_xlabels = []
l2_xlabels = []
l3_xlabels = []
for i, name in enumerate(samplenames):
    sp = name.split(" ")
    l1 = " ".join(sp[-3::])
    l2 = " ".join(sp[0:-3])
    if l2 == '':
        l2 = "NTC"
    l3 = "}" + " ".join(sp[-3:-1])
    if l3 == '}NTC (TE)':
        l3 = "}NTC   "
    l1_xlabels.append(l1)
    l2_xlabels.append(l2)
    if not l3 in l3_xlabels:
        l3_xlabels.append(l3)
    if l2 in posdict.keys():
        posdict[l2].append(xrange[i])
    else:    
        posdict.update({l2: [xrange[i]]})

spec_pos = []
spec_label = []
for key in posdict.keys():
    if key == 'NTC':
        label = key
    else:
        l = key.split(" ")
        if "subsp" in key:
            label=r'$' + abbrev_dict[l[0]] + '$ ' + r'$' + l[1] + '$ ' + r'$' + l[2] + '$ ' + r'$' + l[3] + '$'
        else:
            label=r'$' + abbrev_dict[l[0]] + '$ ' + r'$' + l[1] + '$'

    spec_label.append(label)
    pos = (posdict[key][0] + posdict[key][-1])/2
    spec_pos.append(pos)
  
fig2 = plt.figure() 
spec2 = gridspec.GridSpec(ncols=3, nrows=1, width_ratios=[20, 0.02, 1.5], figure=fig2, wspace=0)
f2_ax1 = fig2.add_subplot(spec2[0, 0])
f2_ax2 = fig2.add_subplot(spec2[0, 2])

img = mpimg.imread('rawdata/bm_rawcqheatmap.png')
legend = mpimg.imread('rawdata/bm_cq_legend.png')

f2_ax1.imshow(img, interpolation='none', extent=[0,1191, 505,0])
f2_ax2.imshow(legend, interpolation='none')

# Plot 1 ticks
f2_ax1.set_xticks(x_ticks)
f2_ax1.set_xticklabels(l3_xlabels, rotation=90, fontsize=4, fontdict={'verticalalignment': 'bottom'})
f2_ax1.tick_params(axis="x", which="major", length=0, width=0, labeltop=True)
# Y-axis for range
f2_ax1.set_yticks(yticks)
f2_ax1.set_yticklabels(len(newylabels) * '{', fontsize=7)
f2_ax1.tick_params(axis="y", which="major", length=0, width=0., pad=0.5)

# second y-axis with assay label
f2_ax1_3 = f2_ax1.twinx()
f2_ax1_3.set_xlim(f2_ax1.get_xlim())
f2_ax1_3.set_ylim(f2_ax1.get_ylim())
f2_ax1_3.yaxis.set_ticks_position("left")
f2_ax1_3.yaxis.set_label_position("left")
f2_ax1_3.spines["left"].set_position(("axes", -0.02))
f2_ax1_3.set_frame_on(False)
f2_ax1_3.patch.set_visible(False)
f2_ax1_3.set_yticks(yticks)
f2_ax1_3.set_yticklabels(newylabels, fontsize=4, style='italic')
f2_ax1_3.tick_params(axis="y", which="major", length=0, width=0., pad=-2)

# Configuration of second x-axis (Species names)
f2_ax1_2 = f2_ax1.twiny()
f2_ax1_2.set_xlim(f2_ax1.get_xlim())
f2_ax1_2.set_ylim(f2_ax1.get_ylim())
f2_ax1_2.spines["top"].set_position(("axes", 1.185))
f2_ax1_2.set_frame_on(False)
f2_ax1_2.patch.set_visible(False)

# Second x-axis ticks
# Species Labels
f2_ax1_2.set_xticks(spec_pos, minor=False)
f2_ax1_2.set_xticklabels(spec_label, rotation=90, fontsize=4, ha="center")
for sp in f2_ax1_2.spines.values():
    sp.set_visible(False)

# Draw "brackets" for species range 
yl = f2_ax1_2.spines["top"].get_position()[1]
minorticks = []
for key in posdict.keys():
    mi = posdict[key][0]
    ma = posdict[key][-1]
    x = [mi, ma]
    y = [yl, yl]
    minorticks.append(mi)
    minorticks.append(ma)
    trans = f2_ax1_2.get_xaxis_transform()
    f2_ax1_2.plot(x, y, color="black", transform=trans, lw=0.5, clip_on=False, label=key)

f2_ax1_2.set_xticks(minorticks, minor=True)
f2_ax1_2.tick_params(axis="x", which="major", length=1, width=0.5, pad=1, direction="out")
f2_ax1_2.tick_params(axis="x", which="minor", length=1.5, width=0.5, pad=0, direction="in")  
    
# General plot 1 configuration
f2_ax1_2.set_xlabel("Pure culture bacterial DNA", weight='bold', fontsize=5, labelpad=1.0)
f2_ax1_3.set_ylabel("High-throughput qPCR primer systems", weight='bold', fontsize=5)
f2_ax1.set_frame_on(True)

#Third x axis
# Configuration of third x-axis (Strainnames)
f2_ax1_4 = f2_ax1.twiny()
f2_ax1_4.set_xlim(f2_ax1.get_xlim())
f2_ax1_4.set_ylim(f2_ax1.get_ylim())
f2_ax1_4.spines["top"].set_position(("axes", 0.982))
f2_ax1_4.set_xticks(x_ticks)
f2_ax1_4.set_xticklabels(l3_xlabels, rotation=90, fontsize=4, fontdict={'verticalalignment': 'bottom'})
f2_ax1_4.tick_params(axis="x", which="major", length=0, width=0, labeltop=True)
for sp in f2_ax1_4.spines.values():
    sp.set_visible(False)

# Remove ticks from legend
f2_ax2.set_title("Cq value", fontsize=4, weight='bold', pad=1.5)
f2_ax2.set_xticks([])
f2_ax2.set_yticks([])
f2_ax1.set_xticks([])
for sp in f2_ax2.spines.values():
    sp.set_linewidth(0.5)

# Save Figure
f2_ax1.axis('tight')
fig2.set_size_inches(7, 4)
if not os.path.isdir("Validation"):
    os.mkdir("Validation")
plt.savefig("Validation/biomark_validation_rawdata.png", dpi=600, bbox_inches='tight')

In [ ]:
# Annotate BioMark melting temperature 

# Create labels
df = pd.read_csv("labelfiles/samples_and_assay_names_validation.csv")
samplenames = list(df["Sample_Name"].dropna())
assaynames = list(df["Assay_Name"].dropna())

# add better labels y-axis
assay_dict = get_assay_dict()
ylabel = assaynames 
newylabels = []
replicates = 3
for i, tik in enumerate(ylabel):
    if i % replicates == 0: 
        label = tik.split(" ")[0]
        label = assay_dict[label]
        newylabels.append(label)

xrange = np.arange(5, 1191, 7)
x_ticks = np.arange(9, 1191, 14)
yticks = np.arange(9, 505, 21)

posdict = {}
l1_xlabels = []
l2_xlabels = []
l3_xlabels = []
for i, name in enumerate(samplenames):
    sp = name.split(" ")
    l1 = " ".join(sp[-3::])
    l2 = " ".join(sp[0:-3])
    if l2 == '':
        l2 = "NTC"
    l3 = "}" + " ".join(sp[-3:-1])
    if l3 == '}NTC (TE)':
        l3 = "}NTC   "
    l1_xlabels.append(l1)
    l2_xlabels.append(l2)
    if not l3 in l3_xlabels:
        l3_xlabels.append(l3)
    if l2 in posdict.keys():
        posdict[l2].append(xrange[i])
    else:    
        posdict.update({l2: [xrange[i]]})

spec_pos = []
spec_label = []
for key in posdict.keys():
    if key == 'NTC':
        label = key
    else:
        l = key.split(" ")
        if "subsp" in key:
            label=r'$' + abbrev_dict[l[0]] + '$ ' + r'$' + l[1] + '$ ' + r'$' + l[2] + '$ ' + r'$' + l[3] + '$'
        else:
            label=r'$' + abbrev_dict[l[0]] + '$ ' + r'$' + l[1] + '$'

    spec_label.append(label)
    pos = (posdict[key][0] + posdict[key][-1])/2
    spec_pos.append(pos)
    
fig2 = plt.figure() 
spec2 = gridspec.GridSpec(ncols=3, nrows=1, width_ratios=[20, 0.02, 1.5], figure=fig2, wspace=0)
f2_ax1 = fig2.add_subplot(spec2[0, 0])
f2_ax2 = fig2.add_subplot(spec2[0, 2])

img = mpimg.imread('rawdata/bm_rawtmheatmap.png')
legend = mpimg.imread('rawdata/bm_tm_legend.png')

f2_ax1.imshow(img, interpolation='none', extent=[0,1191, 505,0])
f2_ax2.imshow(legend, interpolation='none')

# Plot 1 ticks
f2_ax1.set_xticks(x_ticks)
f2_ax1.set_xticklabels(l3_xlabels, rotation=90, fontsize=4, fontdict={'verticalalignment': 'bottom'})
f2_ax1.tick_params(axis="x", which="major", length=0, width=0, labeltop=True)
# Y-axis for range
f2_ax1.set_yticks(yticks)
f2_ax1.set_yticklabels(len(newylabels) * '{', fontsize=7)
f2_ax1.tick_params(axis="y", which="major", length=0, width=0., pad=0.5)

# second y-axis with assay label
f2_ax1_3 = f2_ax1.twinx()
f2_ax1_3.set_xlim(f2_ax1.get_xlim())
f2_ax1_3.set_ylim(f2_ax1.get_ylim())
f2_ax1_3.yaxis.set_ticks_position("left")
f2_ax1_3.yaxis.set_label_position("left")
f2_ax1_3.spines["left"].set_position(("axes", -0.02))
f2_ax1_3.set_frame_on(False)
f2_ax1_3.patch.set_visible(False)
f2_ax1_3.set_yticks(yticks)
f2_ax1_3.set_yticklabels(newylabels, fontsize=4, style='italic')
f2_ax1_3.tick_params(axis="y", which="major", length=0, width=0., pad=-2)

# Configuration of second x-axis (Species names)
f2_ax1_2 = f2_ax1.twiny()
f2_ax1_2.set_xlim(f2_ax1.get_xlim())
f2_ax1_2.set_ylim(f2_ax1.get_ylim())
f2_ax1_2.spines["top"].set_position(("axes", 1.185))
f2_ax1_2.set_frame_on(False)
f2_ax1_2.patch.set_visible(False)

# Second x-axis ticks
# Species Labels
f2_ax1_2.set_xticks(spec_pos, minor=False)
f2_ax1_2.set_xticklabels(spec_label, rotation=90, fontsize=4, ha="center")
for sp in f2_ax1_2.spines.values():
    sp.set_visible(False)

# Draw "brackets" for species range 
yl = f2_ax1_2.spines["top"].get_position()[1]
minorticks = []
for key in posdict.keys():
    mi = posdict[key][0]
    ma = posdict[key][-1]
    x = [mi, ma]
    y = [yl, yl]
    minorticks.append(mi)
    minorticks.append(ma)
    trans = f2_ax1_2.get_xaxis_transform()
    f2_ax1_2.plot(x, y, color="black", transform=trans, lw=0.5, clip_on=False, label=key)

f2_ax1_2.set_xticks(minorticks, minor=True)
f2_ax1_2.tick_params(axis="x", which="major", length=1, width=0.5, pad=1, direction="out")
f2_ax1_2.tick_params(axis="x", which="minor", length=1.5, width=0.5, pad=0, direction="in")  
    
# General plot 1 configuration
f2_ax1_2.set_xlabel("Pure culture bacterial DNA", weight='bold', fontsize=5, labelpad=1.0)
f2_ax1_3.set_ylabel("High-throughput qPCR primer systems", weight='bold', fontsize=5)
f2_ax1.set_frame_on(True)

#Third x axis
# Configuration of third x-axis (Strainnames)
f2_ax1_4 = f2_ax1.twiny()
f2_ax1_4.set_xlim(f2_ax1.get_xlim())
f2_ax1_4.set_ylim(f2_ax1.get_ylim())
f2_ax1_4.spines["top"].set_position(("axes", 0.982))
f2_ax1_4.set_xticks(x_ticks)
f2_ax1_4.set_xticklabels(l3_xlabels, rotation=90, fontsize=4, fontdict={'verticalalignment': 'bottom'})
f2_ax1_4.tick_params(axis="x", which="major", length=0, width=0, labeltop=True)
for sp in f2_ax1_4.spines.values():
    sp.set_visible(False)

# Remove ticks from legend
f2_ax2.set_title("Tm [°C]", fontsize=4, weight='bold', pad=1.5)
f2_ax2.set_xticks([])
f2_ax2.set_yticks([])
f2_ax1.set_xticks([])
for sp in f2_ax2.spines.values():
    sp.set_linewidth(0.5)

# Save Figure
f2_ax1.axis('tight')
fig2.set_size_inches(7, 4)
if not os.path.isdir("Validation"):
    os.mkdir("Validation")
plt.savefig("Validation/biomark_validation_tm_rawdata.png", dpi=600, bbox_inches='tight')

In [ ]:
# Figure Suppl Data
fig = plt.figure(figsize=(8, 11)) 
spec = gridspec.GridSpec(ncols=1, nrows=2, height_ratios=[1,1,], figure=fig, hspace=0)
f_ax1 = fig.add_subplot(spec[0, 0])
f_ax2 = fig.add_subplot(spec[1, 0])


images = [
"Validation/biomark_validation_rawdata.png",
"Validation/biomark_validation_tm_rawdata.png"]

im1 = mpimg.imread(images[0])
im2 = mpimg.imread(images[1])

f_ax1.imshow(im1)
f_ax2.imshow(im2)

f_ax1.set_frame_on(False)
f_ax2.set_frame_on(False)

f_ax1.set_axis_off()
f_ax2.set_axis_off()

f_ax1.axis('tight')
f_ax2.axis('tight')

fig.set_size_inches(7, 8)

f_ax1.text(100, 325, 'A', fontsize=30)
f_ax2.text(100, 325, 'B', fontsize=30)

plt.savefig("Figures/Supplementary_Figure_3.jpg", dpi=600, bbox_inches='tight')

In [ ]:
# Create heatmap data for single strain validation
%run -i biomarkdataparser.py -i rawdata/Assayvalidation_triplicates_rawdata.csv \
--number 3 --replicates assay --outdir Validation/SingleStrains --validation --typefilter NTC Unknown \
-x Mock --rawdata

In [ ]:
# BioMark clean rawdata 
df = pd.read_csv("Validation/SingleStrains/raw_cq_heatmap_data.csv", sep=",")
df = df.set_index(df.columns[0])
newdf = df.T[::-1]

# Sort & Create labels
newdf.sort_index(inplace=True)
assaynames = list(newdf.index.values)
label_df = pd.read_csv("labelfiles/samples_and_assay_names_validation.csv")
label_samplenames = list(label_df["Sample_Name"].dropna())
newdf = newdf[label_samplenames]
samplenames = list(newdf.columns.values)  

# add better labels y-axis
assay_dict = get_assay_dict()
ylabel = assaynames 
newylabels = []
replicates = 3
for i, tik in enumerate(ylabel):
    if i % replicates == 0: 
        label = tik.split(" ")[0]
        label = assay_dict[label]
        newylabels.append(label)

xrange = np.arange(1, len(samplenames)+1)
x_ticks = np.arange(1, len(samplenames)+1, 2)

posdict = {}
l1_xlabels = []
l2_xlabels = []
l3_xlabels = []
for i, name in enumerate(samplenames):
    sp = name.split(" ")
    l1 = " ".join(sp[-3::])
    l2 = " ".join(sp[0:-3])
    if l2 == '':
        l2 = "NTC"
    l3 = "}" + " ".join(sp[-3:-1])
    if l3 == '}NTC (TE)':
        l3 = "}NTC   "
    l1_xlabels.append(l1)
    l2_xlabels.append(l2)
    if not l3 in l3_xlabels:
        l3_xlabels.append(l3)
    if l2 in posdict.keys():
        posdict[l2].append(xrange[i])
    else:
        posdict.update({l2: [xrange[i]]})
        
spec_pos = []
spec_label = []
for key in posdict.keys():
    if key == 'NTC':
        label = key
    else:
        l = key.split(" ")
        if "subsp" in key:
            label=r'$' + abbrev_dict[l[0]] + '$ ' + r'$' + l[1] + '$ ' + r'$' + l[2] + '$ ' + r'$' + l[3] + '$'
        else:
            label=r'$' + abbrev_dict[l[0]] + '$ ' + r'$' + l[1] + '$'

    spec_label.append(label)
    pos = (posdict[key][0] + posdict[key][-1])/2 -0.7
    spec_pos.append(pos)

fig2 = plt.figure() 
spec2 = gridspec.GridSpec(ncols=3, nrows=1, width_ratios=[20, 0.02, 1.5], figure=fig2, wspace=0)
f2_ax1 = fig2.add_subplot(spec2[0, 0])
f2_ax2 = fig2.add_subplot(spec2[0, 2])

cmap = lsc.from_list("", ["yellow", "red", "blue"], 23)
cmap.set_under("black")
cmap.set_over("black")

h = sns.heatmap(
    newdf, vmin=6, vmax=29, cmap=cmap, annot=False,
    annot_kws={"size": 4, "color": "black"}, fmt = '',
    cbar_kws={
            'orientation': 'vertical',
            'extend': 'both', "extendrect": False, "drawedges": True},
    ax=f2_ax1, cbar_ax=f2_ax2, xticklabels=False, yticklabels=False,
    linewidths=0.25, linecolor="black")

colorbar = f2_ax1.collections[0].colorbar
colorbar.set_ticks(np.arange(6.5, 30.5, 1))
colorbar.set_ticklabels([
    "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21",
    "22", "23", "24", "25", "26", "27", "28"])
f2_ax2.invert_yaxis()

# Plot 1 ticks
f2_ax1.set_xticks(x_ticks)
f2_ax1.set_xticklabels(l3_xlabels, rotation=90, fontsize=4, fontdict={'verticalalignment': 'bottom'})
f2_ax1.tick_params(axis="x", which="major", length=0, width=0, labeltop=True)
# Y-axis for range
f2_ax1.set_yticks(np.arange(1, len(assaynames)+1, 1), minor=True)
f2_ax1.set_yticks(np.arange(1.15, len(assaynames)+1, 3))

f2_ax1.set_yticklabels(len(newylabels) * '{', fontsize=6)
f2_ax1.tick_params(axis="y", which="major", length=0, width=0., pad=0.5)
f2_ax1.tick_params(axis="y", which="minor", length=0, width=0., pad=0)

# second y-axis with assay label
f2_ax1_3 = f2_ax1.twinx()
f2_ax1_3.set_xlim(f2_ax1.get_xlim())
f2_ax1_3.set_ylim(f2_ax1.get_ylim())
f2_ax1_3.yaxis.set_ticks_position("left")
f2_ax1_3.yaxis.set_label_position("left")
f2_ax1_3.spines["left"].set_position(("axes", -0.02))
f2_ax1_3.set_frame_on(False)
f2_ax1_3.patch.set_visible(False)
f2_ax1_3.set_yticks(np.arange(1, len(assaynames)+1, 1), minor=True)
f2_ax1_3.set_yticks(np.arange(1, len(assaynames)+1, 3))
f2_ax1_3.set_yticklabels(newylabels, fontsize=4, style='italic')
f2_ax1_3.tick_params(axis="y", which="major", length=0, width=0., pad=-2)
f2_ax1_3.tick_params(axis="y", which="minor", length=0, width=0., pad=0)

# Configuration of second x-axis (Species names)
f2_ax1_2 = f2_ax1.twiny()
f2_ax1_2.set_xlim(f2_ax1.get_xlim())
f2_ax1_2.set_ylim(f2_ax1.get_ylim())
f2_ax1_2.spines["top"].set_position(("axes", 1.2))
f2_ax1_2.set_frame_on(False)
f2_ax1_2.patch.set_visible(False)

# Second x-axis ticks
# Species Labels
f2_ax1_2.set_xticks(spec_pos, minor=False)
f2_ax1_2.set_xticklabels(spec_label, rotation=90, fontsize=4, ha="center")
for sp in f2_ax1_2.spines.values():
    sp.set_visible(False)

# Draw "brackets" for species range 
yl = f2_ax1_2.spines["top"].get_position()[1]
minorticks = []
for key in posdict.keys():
    mi = posdict[key][0] -0.7
    ma = posdict[key][-1] -0.7
    x = [mi, ma]
    y = [yl, yl]
    minorticks.append(mi)
    minorticks.append(ma)
    trans = f2_ax1_2.get_xaxis_transform()
    f2_ax1_2.plot(x, y, color="black", transform=trans, lw=0.5, clip_on=False, label=key)

f2_ax1_2.set_xticks(minorticks, minor=True)
f2_ax1_2.tick_params(axis="x", which="major", length=1, width=0.5, pad=1, direction="out")
f2_ax1_2.tick_params(axis="x", which="minor", length=1.5, width=0.5, pad=0, direction="in")  
    
# General plot 1 configuration
f2_ax1_2.set_xlabel("Pure culture bacterial DNA", weight='bold', fontsize=5, labelpad=1.0)
f2_ax1_3.set_ylabel("High-throughput qPCR primer systems", weight='bold', fontsize=5)
f2_ax1.set_frame_on(True)
for sp in f2_ax1.spines.values():
    sp.set_linewidth(1.5)

#Third x axis
# Configuration of third x-axis (Strainnames)
f2_ax1_4 = f2_ax1.twiny()
f2_ax1_4.set_xlim(f2_ax1.get_xlim())
f2_ax1_4.set_ylim(f2_ax1.get_ylim())
f2_ax1_4.spines["top"].set_position(("axes", 0.979))
f2_ax1_4.set_xticks(x_ticks)
f2_ax1_4.set_xticklabels(l3_xlabels, rotation=90, fontsize=4, fontdict={'verticalalignment': 'bottom'})
f2_ax1_4.tick_params(axis="x", which="major", length=0, width=0, labeltop=True)
for sp in f2_ax1_4.spines.values():
    sp.set_visible(False)

# Remove ticks from legend
f2_ax2.set_title("Cq value", fontsize=4, weight='bold', pad=1.5)
f2_ax2.tick_params(axis="y", length=0, width=0, labelsize=4, pad=-12, direction="in", labelleft=True, labelright=False)
f2_ax1.set_xticks([])
f2_ax1.set_xlabel("")
f2_ax2.set_frame_on(True)
for sp in f2_ax2.spines.values():
    sp.set_linewidth(0.5)

# Save Figure
f2_ax1.axis('tight')
fig2.set_size_inches(7, 4)

plt.savefig("Figures/Supplementary_Figure_4.jpg", dpi=600, bbox_inches='tight')

In [ ]:
# BioMark clean data
df = pd.read_csv("Validation/SingleStrains/cq_heatmap_data.csv", sep=",")
df = df.set_index(df.columns[0])
newdf = df.T[::-1]

# Sort & Create labels
newdf.sort_index(inplace=True)
assaynames = list(newdf.index.values)
label_df = pd.read_csv("labelfiles/samples_and_assay_names_validation.csv")
label_samplenames = list(label_df["Sample_Name"].dropna())
newdf = newdf[label_samplenames]
samplenames = list(newdf.columns.values)  

# add better labels y-axis
assay_dict = get_assay_dict()
ylabel = assaynames 
newylabels = []

for i, tik in enumerate(ylabel):
    label = tik.split(" ")[0]
    label = assay_dict[label]
    newylabels.append(label)

xrange = np.arange(1, len(samplenames)+1)
x_ticks = np.arange(1, len(samplenames)+1, 2)

posdict = {}
l1_xlabels = []
l2_xlabels = []
l3_xlabels = []
for i, name in enumerate(samplenames):
    sp = name.split(" ")
    l1 = " ".join(sp[-3::])
    l2 = " ".join(sp[0:-3])
    if l2 == '':
        l2 = "NTC"
    l3 = "}" + " ".join(sp[-3:-1])
    if l3 == '}NTC (TE)':
        l3 = "}NTC   "
    l1_xlabels.append(l1)
    l2_xlabels.append(l2)
    if not l3 in l3_xlabels:
        l3_xlabels.append(l3)
    if l2 in posdict.keys():
        posdict[l2].append(xrange[i])
    else:
        posdict.update({l2: [xrange[i]]})
        
spec_pos = []
spec_label = []
for key in posdict.keys():
    if key == 'NTC':
        label = key
    else:
        l = key.split(" ")
        if "subsp" in key:
            label=r'$' + abbrev_dict[l[0]] + '$ ' + r'$' + l[1] + '$ ' + r'$' + l[2] + '$ ' + r'$' + l[3] + '$'
        else:
            label=r'$' + abbrev_dict[l[0]] + '$ ' + r'$' + l[1] + '$'

    spec_label.append(label)
    pos = (posdict[key][0] + posdict[key][-1])/2 -0.7
    spec_pos.append(pos)

fig2 = plt.figure() 
spec2 = gridspec.GridSpec(ncols=3, nrows=1, width_ratios=[20, 0.02, 1.5], figure=fig2, wspace=0)
f2_ax1 = fig2.add_subplot(spec2[0, 0])
f2_ax2 = fig2.add_subplot(spec2[0, 2])

cmap = lsc.from_list("", ["yellow", "red", "blue"], 23)
cmap.set_under("black")
cmap.set_over("black")
cmap.set_bad("black")

h = sns.heatmap(
    newdf, vmin=6, vmax=29, cmap=cmap, annot=False,
    annot_kws={"size": 4, "color": "black"}, fmt = "",
    cbar_kws={
            'orientation': 'vertical',
            'extend': 'both', "extendrect": False, "drawedges": True},
    ax=f2_ax1, cbar_ax=f2_ax2, xticklabels=False, yticklabels=False,
    linewidths=0.25, linecolor="black")

colorbar = f2_ax1.collections[0].colorbar
colorbar.set_ticks(np.arange(6.5, 29.5, 1))
colorbar.set_ticklabels([
    "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21",
    "22", "23", "24", "25", "26", "27", "28"])
f2_ax2.invert_yaxis()

# Plot 1 ticks
f2_ax1.set_xticks(x_ticks)
f2_ax1.set_xticklabels(l3_xlabels, rotation=90, fontsize=4, fontdict={'verticalalignment': 'bottom'})
f2_ax1.tick_params(axis="x", which="major", length=0, width=0, labeltop=True)

# Y-axis
f2_ax1.set_yticks(np.arange(0.5, len(assaynames)+0.5, 1))
f2_ax1.set_yticklabels(newylabels, fontsize=4, style='italic')
f2_ax1.tick_params(axis="y", which="major", length=0, width=0., pad=2)

# second y-axis with assay label
f2_ax1_3 = f2_ax1.twinx()
f2_ax1_3.set_xlim(f2_ax1.get_xlim())
f2_ax1_3.set_ylim(f2_ax1.get_ylim())
f2_ax1_3.set_frame_on(True)
f2_ax1_3.patch.set_visible(False)
f2_ax1_3.set_yticks([])
f2_ax1_3.tick_params(axis="y", which="major", length=0, width=0., pad=-2)
for sp in f2_ax1_3.spines.values():
    sp.set_linewidth(0.5)

# Configuration of second x-axis (Species names)
f2_ax1_2 = f2_ax1.twiny()
f2_ax1_2.set_xlim(f2_ax1.get_xlim())
f2_ax1_2.set_ylim(f2_ax1.get_ylim())
f2_ax1_2.spines["top"].set_position(("axes", 1.2))
f2_ax1_2.set_frame_on(False)
f2_ax1_2.patch.set_visible(False)

# Second x-axis ticks
# Species Labels
f2_ax1_2.set_xticks(spec_pos, minor=False)
f2_ax1_2.set_xticklabels(spec_label, rotation=90, fontsize=4, ha="center")
#for sp in f2_ax1_2.spines.values():
    #sp.set_visible(False)

# Draw "brackets" for species range 
yl = f2_ax1_2.spines["top"].get_position()[1]
minorticks = []
for key in posdict.keys():
    mi = posdict[key][0] -0.7
    ma = posdict[key][-1] -0.7
    x = [mi, ma]
    y = [yl, yl]
    minorticks.append(mi)
    minorticks.append(ma)
    trans = f2_ax1_2.get_xaxis_transform()
    f2_ax1_2.plot(x, y, color="black", transform=trans, lw=0.5, clip_on=False, label=key)

f2_ax1_2.set_xticks(minorticks, minor=True)
f2_ax1_2.tick_params(axis="x", which="major", length=1, width=0.5, pad=1, direction="out")
f2_ax1_2.tick_params(axis="x", which="minor", length=1.5, width=0.5, pad=0, direction="in")  
    
# General plot 1 configuration
f2_ax1_2.set_xlabel("Pure culture bacterial DNA", weight='bold', fontsize=5, labelpad=1.0)
f2_ax1.set_ylabel("High-throughput qPCR primer systems", weight='bold', fontsize=5)
f2_ax1.set_frame_on(True)
for sp in f2_ax1.spines.values():
    sp.set_linewidth(1.5)

#Third x axis
# Configuration of third x-axis (Strainnames)
f2_ax1_4 = f2_ax1.twiny()
f2_ax1_4.set_xlim(f2_ax1.get_xlim())
f2_ax1_4.set_ylim(f2_ax1.get_ylim())
f2_ax1_4.spines["top"].set_position(("axes", 0.979))
f2_ax1_4.set_xticks(x_ticks)
f2_ax1_4.set_xticklabels(l3_xlabels, rotation=90, fontsize=4, fontdict={'verticalalignment': 'bottom'})
f2_ax1_4.tick_params(axis="x", which="major", length=0, width=0, labeltop=True)
for sp in f2_ax1_4.spines.values():
    sp.set_visible(False)

# Remove ticks from legend
f2_ax2.set_title("Cq value", fontsize=4, weight='bold', pad=1.5)
f2_ax2.tick_params(axis="y", length=0, width=0, labelsize=4, pad=-12, direction="in", labelleft=True, labelright=False)
f2_ax1.set_xticks([])
f2_ax1.set_xlabel("")
f2_ax2.set_frame_on(True)
for sp in f2_ax2.spines.values():
    sp.set_linewidth(0.5)

# Save Figure
f2_ax1.axis('tight')
fig2.set_size_inches(7, 4)

plt.savefig("Figures/Figure_2.jpg", dpi=600, bbox_inches='tight')